In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import os

from langchain.chains.question_answering import load_qa_chain
from langchain_openai import ChatOpenAI

from langchain.globals import set_llm_cache
from langchain.cache import SQLiteCache

set_llm_cache(SQLiteCache(database_path="data/.langchain.db"))

def create_agent_chain(llm):
    chain = load_qa_chain(llm, chain_type="stuff")
    return chain

llm = ChatOpenAI(model="gpt-4o-mini")
bigllm = ChatOpenAI(model="gpt-4o")
chain = create_agent_chain(llm) #cached_underlying_embeddings_chroma

def get_llm_response(query,vectordb,temperature=0.1,k=10):
    matching_docs = vectordb.similarity_search(query,k)
    answer = chain.run(input_documents=matching_docs, question=query)

    return answer, matching_docs

df = pd.read_parquet("data/consolidated.parquet.gzip")

STYLE = """* Master the Art of Footnotes: Treat them as your secret weapon. Use them to share amusing observations that pop into your head while telling the story. These asides should feel like you're whispering a clever joke to your reader without derailing the main narrative.
* Embrace the Pun-possible: Keep a sharp eye out for common phrases that could be twisted or taken literally in unexpected ways. If you spot a potential wordplay, don't resist it – Pratchett certainly wouldn't. But remember, the best puns aren't just groaners; they should also illuminate something about your story or characters.
* Wield Your Metaphors Like a Slightly Wobbly Sword: Craft comparisons that are simultaneously ridiculous and perfect. For example, don't just say someone is confused – say their thoughts are "as organized as a library after a tornado had decided to learn the Dewey Decimal System."
* Narrate Like a Clever Friend: Write as if you're telling the story to someone smart over a cup of tea, occasionally pausing to point out the absurdity of what you're describing. Let your narrator be wise, witty, and slightly world-weary, but never cynical.
* Turn Philosophy into Comedy: Take big, serious ideas – death, justice, belief, power – and examine them through the lens of everyday situations or minor absurdities. If you're writing about the nature of belief, maybe explore it through a character who accidentally starts a religion while trying to order pizza.
* Build Characters Who Are Sensibly Insane: Create people who operate on perfectly logical principles that just happen to be slightly askew from everyone else's reality. Their actions should make complete sense to them, even if they seem bizarre to others.
* Remember Pratchett's golden rule: the humor should serve the story, not the other way around. Each joke, pun, or amusing footnote should help reveal something about your world or characters.

AND THE MOST IMPORTANT

* Use information from the summaries to provide the reader with some actual insights into what the future might yield =)

"""


/tmp/ipykernel_156923/3859778177.py:13: LangChainDeprecationWarning: This class is deprecated. See the following migration guides for replacements based on `chain_type`:
stuff: https://python.langchain.com/docs/versions/migrating_chains/stuff_docs_chain
map_reduce: https://python.langchain.com/docs/versions/migrating_chains/map_reduce_chain
refine: https://python.langchain.com/docs/versions/migrating_chains/refine_chain
map_rerank: https://python.langchain.com/docs/versions/migrating_chains/map_rerank_docs_chain

See also guides on retrieval and question-answering here: https://python.langchain.com/docs/how_to/#qa-with-rag
  chain = load_qa_chain(llm, chain_type="stuff")


In [3]:
df = pd.read_parquet("data/consolidated.parquet.gzip")
df

,src,content,LEN,title,keywords,themes,summary,origin,url
0,c3e8d6ff1b93d47f455b0e6cc885f181,The mysterious reported sightings of drone clu...,4083,Mystery Drone Sightings Frustrate Officials,"[drone sightings, New Jersey, Congress, FBI, s...","[government response, public safety, drone tec...",The recent reports of mysterious drone cluster...,20241229.md,b'https://www.theguardian.com/us-news/2024/dec...
1,eefdc09d71c548a9343dffe432839de6,A tiny cereal grain from Ethiopia and Eritrea ...,2672,Teff: The Sustainable Ancient Grain,"[teff, gluten-free, Ethiopia, sustainable, mar...","[Food & Nutrition, Agriculture, Sustainability]","Teff, a tiny grain from Ethiopia and Eritrea, ...",20241222.md,b'https://www.fooddive.com/news/ingredients-in...
2,faa0e4de05145c04c5d22a8449a70d95,Continuing improvements in AI could trigger a ...,2408,The Future of Work with AI,"[AI, wages, growth, employment, adaptation, hu...","[Economics, Technology, Labor Markets]",The text discusses the potential impact of adv...,20241222.md,b'https://www.exponentialview.co/p/are-we-faci...
3,7ec50abadcf7e1ee9bf56bd5c5b15467,Major health insurers are deleting images of t...,5641,Insurers Remove Executive Images After Shooting,"[UnitedHealthcare, CEO, Brian Thompson, health...","[Healthcare, Corporate Governance, Social Medi...",The recent assassination of UnitedHealthcare C...,20241229.md,b'https://www.healthcaredive.com/news/health-i...
4,b6a1403e12fe78cbb944ee7233c6d1f4,We live in an Age of Uncertainty. Not just bec...,6491,Embracing Uncertainty in Life,"[uncertainty, predictions, future, forecasters...","[psychology, risk management, philosophy]",The text discusses the pervasive nature of unc...,20241229.md,b'https://www.wired.com/story/embrace-uncertai...
...,...,...,...,...,...,...,...,...,...
2183,edbb7336dfb26098bcc966ca17c074b9,It was time to consider calling it a night. Sh...,15656,The Radical Power of Hanging Out,"[hanging out, book, Sheila Liming, crisis, iso...","[socializing, isolation, technology]","The text revolves around the concept of ""hangi...",20230305,b'https://slate.com/culture/2023/02/hanging-ou...
2184,3fbb11d6e949d1e662aa6a146bb6cda0,China’s growing number of insomniacs are turni...,2859,China's Insomniacs Turn to Sleeping Assistants,"[insomniacs, sleeping assistants, Taobao, Chin...","[Insomnia, Sleeping services, Sleep economy]","China has a growing number of insomniacs, lead...",20221031,b'https://www.sixthtone.com/news/1011417?utm_s...
2185,e8b7faf708568f9f39abe04b778c4631,I would say my efficiency is up ~20% since sta...,14337,The Impact of ChatGPT on Efficiency and Inform...,"[efficiency, Google searches, StackOverflow vi...","[productivity enhancement, automation, relianc...",The text highlights the author's increased eff...,20230521,b'https://news.ycombinator.com/item?id=35846386'
2186,b79a2baa87b68283198416791b93bce4,The U.S. government has restricted sales of Nv...,2125,US Restricts Nvidia GPU Sales to Thwart China'...,"[U.S. government, Nvidia, high-performance com...","[Export regulations, China-US relations, AI de...",The U.S. government has imposed restrictions o...,20230320,b'https://www.tomshardware.com/news/us-bans-sa...


In [4]:

src = [x for x in list(df.origin.unique()) if str(x).endswith(".md")]
stories = []

for s in src:

    REFS = list( df[df.origin == s].summary )
    if len(REFS) > 6:
        context = "* "+ "\n* ".join(REFS)

        prompt  = "## Objective\n\n"
        prompt += "You are a sci-fi writer, and you want to write a short story in one page or slightly less, that takes place in today world and seems very realistic.\n\n"
        prompt += "You are given summaries of texts that contains elements (possibly forward-looking) that can feed this short story. Try and use each of the summaries, especially the 'innovative' parts.\n\n"
        prompt += "You need to pick a theme for that story (which you don't need to tell me) and make it the backbone of the story.\n\nStart and give it a try! Don't forget to add at least a point in the story that comes from each  bullet point, but you can interleave the different points of course. Give me around one page that surprising short story - you don't need to give me any intro to it."
        prompt += "\n\n## How to do it\n\n"+"* Change any reference to personal names so to anonymyse real persons names\n* You will use a style mix of Terry Pratchett and Philip K Dick:\n\n"+STYLE
        prompt += "\n\n## Summaries of texts\n\n"+ context
        story = llm.invoke(prompt).content
        title = llm.invoke("Give a title to the short story below. Don't use quotes or any accompanying text:\n\n"+story).content

        stories.append([s,title,story])

STORY  = pd.DataFrame(stories,columns=["post","title","story"])
STORY


,post,title,story
0,20241229.md,Reflections of Uncertainty,The sun had just dipped below the horizon in V...
1,20241222.md,A Sprinkle of Ambition,"The sun dipped below the horizon, casting a go..."


In [5]:
print("# "+STORY.iloc[0].title+"\n\n"+STORY.iloc[0].story)

# Reflections of Uncertainty

The sun had just dipped below the horizon in Viganella, leaving the town in its habitual three-month embrace of darkness. Yet, instead of the usual gloom, there was a buzz of excitement as the townsfolk gathered in the main square, illuminated by an enormous, glimmering mirror designed to reflect the last rays of the day. It was a festive atmosphere, reminiscent of the time when they had first turned on the mirror—a triumph against nature, a beacon of hope in a world too often shrouded in uncertainty. 

At the center of this gathering stood a peculiar figure: a self-styled futurist named Quinn, who was convinced that the drones reported over New Jersey were sent by intergalactic insurance agents surveying Earth for potential high-risk clients. "Have you seen the state of our healthcare system?" Quinn quipped to an amused crowd, "They must be taking notes!" *Footnote: It’s a little-known fact that insurance agents are the real space cowboys, roaming the cos

In [6]:
import pandas as pd

In [7]:
from writers import getStories
stories = getStories()

In [8]:
stories

,post,title,story
0,20241229.md,Reflections of Uncertainty,The sun had just dipped below the horizon in V...
1,20241222.md,A Sprinkle of Ambition,"The sun dipped below the horizon, casting a go..."
2,20241207,Building a Future: The Balance of Man and Machine,The sun hung lazily over the shimmering skylin...
3,20241208,Navigating the Digital Tightrope,"The conference room was bustling, a kaleidosco..."
4,20241215,The Polystyrene-Eating Heroes of Widdershins,"In the quaint town of Widdershins, where smart..."
...,...,...,...
116,20220828,Digital Derinkuyu: Navigating the Labyrinth of...,The sun was setting over the burgeoning metrop...
117,20221117,Digital Renaissance: A Tapestry of Absurdity,In a world where the stock image industry was ...
118,20230819,Digital Reflections in an Absurd World,"Evelyn stared at her phone, a device that had ..."
119,20221228,Reclaiming Authenticity,In a world where everything was as available a...


# TODO

In [9]:
story = str(stories[stories.post == "20241229.md"].iloc[0].story)
title = str(stories[stories.post == "20241229.md"].iloc[0].title)
print("# ",title,"\n")
print(story)

#  Reflections of Uncertainty 

The sun had just dipped below the horizon in Viganella, leaving the town in its habitual three-month embrace of darkness. Yet, instead of the usual gloom, there was a buzz of excitement as the townsfolk gathered in the main square, illuminated by an enormous, glimmering mirror designed to reflect the last rays of the day. It was a festive atmosphere, reminiscent of the time when they had first turned on the mirror—a triumph against nature, a beacon of hope in a world too often shrouded in uncertainty. 

At the center of this gathering stood a peculiar figure: a self-styled futurist named Quinn, who was convinced that the drones reported over New Jersey were sent by intergalactic insurance agents surveying Earth for potential high-risk clients. "Have you seen the state of our healthcare system?" Quinn quipped to an amused crowd, "They must be taking notes!" *Footnote: It’s a little-known fact that insurance agents are the real space cowboys, roaming the c

# 2024

In [13]:
df = df.fillna("na")
four = df[df.origin.str.startswith("2024")]
four

,src,content,LEN,title,keywords,themes,summary,origin,url
0,c3e8d6ff1b93d47f455b0e6cc885f181,The mysterious reported sightings of drone clu...,4083,Mystery Drone Sightings Frustrate Officials,"[drone sightings, New Jersey, Congress, FBI, s...","[government response, public safety, drone tec...",The recent reports of mysterious drone cluster...,20241229.md,b'https://www.theguardian.com/us-news/2024/dec...
1,eefdc09d71c548a9343dffe432839de6,A tiny cereal grain from Ethiopia and Eritrea ...,2672,Teff: The Sustainable Ancient Grain,"[teff, gluten-free, Ethiopia, sustainable, mar...","[Food & Nutrition, Agriculture, Sustainability]","Teff, a tiny grain from Ethiopia and Eritrea, ...",20241222.md,b'https://www.fooddive.com/news/ingredients-in...
2,faa0e4de05145c04c5d22a8449a70d95,Continuing improvements in AI could trigger a ...,2408,The Future of Work with AI,"[AI, wages, growth, employment, adaptation, hu...","[Economics, Technology, Labor Markets]",The text discusses the potential impact of adv...,20241222.md,b'https://www.exponentialview.co/p/are-we-faci...
3,7ec50abadcf7e1ee9bf56bd5c5b15467,Major health insurers are deleting images of t...,5641,Insurers Remove Executive Images After Shooting,"[UnitedHealthcare, CEO, Brian Thompson, health...","[Healthcare, Corporate Governance, Social Medi...",The recent assassination of UnitedHealthcare C...,20241229.md,b'https://www.healthcaredive.com/news/health-i...
4,b6a1403e12fe78cbb944ee7233c6d1f4,We live in an Age of Uncertainty. Not just bec...,6491,Embracing Uncertainty in Life,"[uncertainty, predictions, future, forecasters...","[psychology, risk management, philosophy]",The text discusses the pervasive nature of unc...,20241229.md,b'https://www.wired.com/story/embrace-uncertai...
...,...,...,...,...,...,...,...,...,...
922,3c5b8213e451f2e4acb9f79e1cdfefef,Mapping Middle-earth: The lopsided demographic...,9159,Mapping Middle-earth: The lopsided demographic...,"[Middle-earth, demographics, Tolkien, world, g...","[Fantasy literature, Gender imbalance, Life ex...","Middle-earth, the fictional universe created b...",20240121,b'https://bigthink.com/strange-maps/middle-ear...
923,90de84f63a10fdb941b767bf3f8ba615,Anjali Goswami thinks cats are perfect—not in ...,6873,The Evolutionary Perfection of Cats,"[cats, evolution, evolutionary biologist, spec...","[Evolutionary Biology, Specialization in Cats,...","Anjali Goswami, an evolutionary biologist, arg...",20240121,b'https://www.scientificamerican.com/article/c...
924,086938a3a1c7de2af0524c025ae6aab5,Many of you have probably seen the famous adve...,5289,The Myth of Shackleton's Famous Advertisement,"[advertisement, Ernest Shackleton, Endurance e...","[advertising, exploration, myth]",The famous advertisement attributed to Ernest ...,20240121,b'https://discerninghistory.com/2013/05/shackl...
926,a438591373b3ce215c807a83c53eabe3,Designing the future using science fiction\n“T...,6610,Designing the Future through Science Fiction,"[future, science fiction, visions, narratives,...","[Envisioning the future, Impact of technology ...",The text discusses the importance of using sci...,20240121,b'https://www-wondermash-eu.cdn.ampproject.org...


In [20]:
reviews = four.explode("themes")[["title","content","summary","LEN","themes"]].reset_index(drop=True)
reviews["themes"] = reviews["themes"].str.title()
reviews

,title,content,summary,LEN,themes
0,Mystery Drone Sightings Frustrate Officials,The mysterious reported sightings of drone clu...,The recent reports of mysterious drone cluster...,4083,Government Response
1,Mystery Drone Sightings Frustrate Officials,The mysterious reported sightings of drone clu...,The recent reports of mysterious drone cluster...,4083,Public Safety
2,Mystery Drone Sightings Frustrate Officials,The mysterious reported sightings of drone clu...,The recent reports of mysterious drone cluster...,4083,Drone Technology
3,Teff: The Sustainable Ancient Grain,A tiny cereal grain from Ethiopia and Eritrea ...,"Teff, a tiny grain from Ethiopia and Eritrea, ...",2672,Food & Nutrition
4,Teff: The Sustainable Ancient Grain,A tiny cereal grain from Ethiopia and Eritrea ...,"Teff, a tiny grain from Ethiopia and Eritrea, ...",2672,Agriculture
...,...,...,...,...,...
2601,Designing the Future through Science Fiction,Designing the future using science fiction\n“T...,The text discusses the importance of using sci...,6610,Impact Of Technology On Society
2602,Designing the Future through Science Fiction,Designing the future using science fiction\n“T...,The text discusses the importance of using sci...,6610,Sustainability And Resilience
2603,How to Fall Down a Rabbit Hole,How to Fall Down a Rabbit Hole\nby Alden Burke...,"""How to Fall Down a Rabbit Hole"" is a guide th...",9906,Curiosity
2604,How to Fall Down a Rabbit Hole,How to Fall Down a Rabbit Hole\nby Alden Burke...,"""How to Fall Down a Rabbit Hole"" is a guide th...",9906,Self-Exploration


In [30]:
rv = reviews.groupby("themes").count().reset_index(drop=False)[["themes","content"]].sort_values(by="content",ascending=False)[:20]
rv

,themes,content
173,Artificial Intelligence,86
1694,Technology,53
487,Cybersecurity,27
339,Climate Change,21
1662,Sustainability,15
1151,Mental Health,13
1000,Innovation,13
1019,International Relations,13
1087,Literature,11
503,Data Privacy,11
